<a href="https://colab.research.google.com/github/rohitrv8922/Speech-Emotion-Recognition-Detection/blob/main/Speech_Emotion_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EDA

In [25]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Project Introduction

Import Libraries

In [26]:
# Importing all the required Libraries

import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential                                                            
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D,MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization,GlobalMaxPool2D,Activation,GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from matplotlib.image import imread
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from tensorflow.keras import Model
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers
import numpy as np 
import pandas as pd
import cv2
import os
from sklearn.metrics import confusion_matrix
import itertools
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.8.2


In [31]:
#Import the libraries
import zipfile
import os
#Opens the zip file in read mode
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Dataset/icml_face_data.csv.zip', 'r')
#Extracts the files in folder
zip_ref.extractall('/content/drive/MyDrive/Dataset') 
zip_ref.close()

In [42]:
# Defining train and validation path
train_path = '/content/drive/MyDrive/Dataset/train.csv'
val_path = '/o-ntent/drive/MyDrive/Dataset/validation'

In [44]:
# Training images
total_train_images = 0
for dir_ in os.listdir(train_path):
    count = 0
    for f in os.listdir(train_path+'/' + dir_+"/"):
        count += 1
        total_train_images += 1
    print(f"{dir_} has {count} number of images")
    
print(f"\ntotal train images are {total_train_images}")

FileNotFoundError: ignored

# New Section